
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 5 - RL <br>
<font color=696880 size=4>
Amirhossein Abedi


____

# Personal Data

In [1]:
# Set your student number and name
student_number = '401105815'
Name = 'Radin'
Last_Name = 'Cheraghi'

# <center>تاکسی</center>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با محیط Taxi در gymnasium کار کنیم.
<br>
در اینجا در نقش یک تاکسی هستیم که باید یک مسافر را از یک مکان مشخص برداریم و در مقصد موردنظر او پیاده کنیم. توضیحات تکمیلی درمورد این محیط را می‌توانید در  <a href="https://gymnasium.farama.org/environments/toy_text/taxi/">Taxi</a>  پیدا کنید.

![Taxi](taxi.png)
</div>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با پیاده‌سازی الگوریتم value iteration بهترین پاسخ در هر حالت را پیدا کنیم. هر حالت در این بازی با یک عدد بین ۰ تا ۴۹۹ نشان داده می‌شود. که فرمول آن به صورت زیر است
<br><br>

### <center dir="ltr">state = ((5 × row + col) × 5 + passenger_location) × 4 + destination</center>

<br><br>
مقادیر row و col که مشخص است و عددی بین ۰ تا ۴ است. مقدار passenger_location بین ۰ تا ۴ می‌باشد که هرکدام از اعداد نشان می‌دهد که مسافر در کدام پایانه قرار دارد. پایانه‌ها مکان‌هایی مشخص درون بازی هستند که می‌تواند با استفاده از تابع env.render آن‌ها را مشاهده کنید. ۴ پایانه به رنگ‌های قرمز، سبز، زرد، آبی درون بازی قرار دارند که مقادیر ۰ تا ۳ به ترتیب همین پایانه‌ها را نشان می‌دهد. و در نهایت مقدار ۴ برای passenger_location به این معنا است که مسافر در تاکسی قرار دارد.
<br><br>
مقدار destination نشان می‌دهد که مسافر می‌خواهد در کدام پایانه پیاده شود. نگاشت عدد و رنگ‌ها مانند بخش passenger_location می‌باشد و فقط مقادیر ۰ تا ۳ را به خود می‌گیرد.
<br><br>
با هر بار اجرای تابع reset و step بر روی env ، مقدار اولی که به شما برگردانده می‌شود مقدار state است. در ادامه به action ها می‌پردازیم. در کل ۶ اکشن مختلف می‌توانیم اجرا کنیم. اکشن ۰ تا ۳ به ترتیب مربوط به حرکات پایین، بالا، راست و چپ است. حرکت ۴ مربوط به برداشتن مسافر و حرکت ۵ پیاده کردن مسافر است. استفاده از حرکت ۴ و ۵ هنگامی که مجاز نباشند، ریوارد منفی زیادی خواهد داشت.
</div>

In [3]:
!pip install numpy
!pip install gymnasium

In [19]:
import gymnasium as gym
env = gym.make('Taxi-v3', render_mode='ansi')

<div dir="rtl" style="font-size:18px">
خروجی تابع reset شامل یک ۲ تایی است. اولین خروجی آن یک عدد است که نشان‌دهنده حالت اولیه می‌باشد و خروجی دوم یک دیکشنری است که مقدار action_mask شامل تمام اکشن‌هایی است که در این حالت می‌توانید انجام دهید.
</div>

In [20]:
env.reset()

(372, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})

<div dir="rtl" style="font-size:18px">
خروجی تابع step هم تا حدود زیادی شبیه به reset می باشد. با این تفاوت که بعد از خروجی حالت، یک عدد و ۲ بولین به شما می‌دهد که عدد اول مقدار reward و بولین اول تمام شدن بازی را مشخص می‌کند.
</div>

<div dir="rtl" style="font-size:18px">
در ادامه کلاس VI را پیاده‌سازی کنید که با گرفتن discount factor الگوریتم value iteration را پیاده‌سازی می‌کند.
</div>

In [21]:
env.step(0)

(472,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 1, 0], dtype=int8)})

In [23]:
import numpy as np

class VI():
    def __init__(self, gamma: float = 0.95):
        # todo
        self.values = np.zeros(500)
        self.gamma = gamma
        self.learn()

    def policy(self, state: int) -> int:
      set1 = [(0, 1), (1, 1), (3, 0), (4, 0), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (3, 2), (4, 2)]
      set2 = [(0, 2), (1, 2), (3, 1), (4, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (3, 3), (4, 3)]
      max_v = float('-inf')
      for action in range(6):
        reward = self.get_reward(state, action)
        row, col, passenger_location, destination = self._num_to_state(state)
        state_val = state
        if action == 0 and row != 4:
          state_val = self._state_to_num(row + 1, col, passenger_location, destination)
        if action == 1 and row != 0:
          state_val = self._state_to_num(row - 1, col, passenger_location, destination)
        if action == 2 and (row, col) not in set1:
          state_val = self._state_to_num(row, col + 1, passenger_location, destination)
        if action == 3 and (row, col) not in set2:
          state_val = self._state_to_num(row, col - 1, passenger_location, destination)
        if action == 4 and reward != -10:
          state_val = self._state_to_num(row, col, 4, destination)
        new_val = reward + self.values[state_val] * self.gamma
        if new_val > max_v:
          max_v = new_val
          index = action
      return index

    def _num_to_state(self, num: int):
        dest = num % 4
        num //= 4
        passenger_loc = num % 5
        num //= 5
        col = num % 5
        num //= 5
        return num, col, passenger_loc, dest

    def _state_to_num(self, row, col, passenger_loc, dest):
        ret = row
        ret = 5 * ret + col
        ret = 5 * ret + passenger_loc
        ret = 4 * ret + dest
        return ret

    def get_reward(self, state, action):
        row, col, passenger_location, destination = self._num_to_state(state)
        reward = 0
        if action == 4:
          reward = 0 # I can change it later
          if passenger_location == 0:
            if row != 0 or col != 0:
              reward = -10 # I can change it in future
          elif passenger_location == 1:
            if row != 0 or col != 4:
              reward = -10 # I can change it in future
          elif passenger_location == 2:
            if row != 4 or col != 0:
              reward = -10 # I can change it in future
          elif passenger_location == 3:
            if row != 4 or col != 3:
              reward = -10 # I can change it in future
        elif action == 5:
          reward = 20 # I can change it later
          if destination == 0:
            if row != 0 or col != 0:
              reward = -10 # I can change it in future
          elif destination == 1:
            if row != 0 or col != 4:
              reward = -10 # I can change it in future
          elif destination == 2:
            if row != 4 or col != 0:
              reward = -10 # I can change it in future
          elif destination == 3:
            if row != 4 or col != 3:
              reward = -10 # I can change it in future
          if passenger_location != 4:
            reward = -10

        return reward

    def learn(self):
      set1 = [(0, 1), (1, 1), (3, 0), (4, 0), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (3, 2), (4, 2)]
      set2 = [(0, 2), (1, 2), (3, 1), (4, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (3, 3), (4, 3)]
      num_iterations = 10000
      for iteration in range(num_iterations):
        tmp = []
        for i in self.values:
          tmp.append(i)
        tmp_values = np.array(tmp)
        for state in range(len(self.values)):
          max_v = float('-inf')
          for action in range(6):
            reward = self.get_reward(state, action)
            row, col, passenger_location, destination = self._num_to_state(state)
            state_val = state
            if action == 0 and row != 4:
             state_val = self._state_to_num(row + 1, col, passenger_location, destination)
            if action == 1 and row != 0:
              state_val = self._state_to_num(row - 1, col, passenger_location, destination)
            if action == 2 and (row, col) not in set1:
              state_val = self._state_to_num(row, col + 1, passenger_location, destination)
            if action == 3 and (row, col) not in set2:
              state_val = self._state_to_num(row, col - 1, passenger_location, destination)
            if action == 4 and reward != -10:
              state_val = self._state_to_num(row, col, 4, destination)
            new_val = reward + self.values[state_val] * self.gamma
            if new_val > max_v:
              max_v = new_val
          tmp_values[state] = max_v
        self.values = tmp_values
        self.values[self._state_to_num(4, 0, 4, 2)] = 1000
        self.values[self._state_to_num(4, 3, 4, 3)] = 1000

        self.values[self._state_to_num(0, 0, 4, 0)] = 1000
        self.values[self._state_to_num(0, 4, 4, 1)] = 1000

agent = VI()

<div dir="rtl" style="font-size:18px">
۲ سلول بعدی را اجرا کنید. در هر نقشه نشان داده شده است که با برقرار بودن شرایط به خصوص، تاکسی به کدام سمت حرکت می‌کند. در انتها در چند خط منطقی عمل کردن مدل خود را بررسی کنید.
</div>

In [24]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for passenger_loc in range(4):
    print(f'Passenger in {terminal[passenger_loc]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, passenger_loc, 1))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')



Passenger in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in G
+---------+
|R:↓|→:→:G|
|↓:↓|↑:↑:↑|
|→:→:↑:↑:↑|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|↑:↑|↓:↓|
|Y|↑:↑|B:←|
+---------+



همانطور که در خروجی‌های بالا مشخص می‌باشد تاکسی همواره در تلاش است تا با توجه به موانع به پایانه‌ای که مسافر در آن مستقر است رسیده و آن را سوار کند که از لحاظ منطقی درست بوده و در خروجی‌ها نیز چنین چیزی دیده می‌شود

In [25]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for dest in range(4):
    print(f'Passenger on board, destination in {terminal[dest]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, 4, dest))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')

Passenger on board, destination in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in G
+---------+
|R:↓|→:→:G|
|↓:↓|↑:↑:↑|
|→:→:↑:↑:↑|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|↑:↑|↓:↓|
|Y|↑:↑|B:←|
+---------+



همانطور که در خروجی‌های بالا مشخص می‌باشد تاکسی همواره در تلاش است تا با توجه به موانع به پایانه‌ای که مسافر قرار است در آن پیاده شود برسد و آن را پیاده کند که از لحاظ منطقی درست بوده و در خروجی‌ها نیز چنین چیزی دیده می‌شود